In [97]:
import pandas as pd
import glob
import os

In [346]:
path = "/Users/sonia/Downloads/Archive/"
os.chdir(path)
fileNames = os.listdir(path)

df = pd.ExcelFile("/Users/sonia/Downloads/hazardousFrame.xlsx")
sheetNames = df.sheet_names
df = df.parse(sheetNames[0])
df.at[9,'endFrame'] = 410
df.at[6,'endFrame'] = 430
df.at[0,'endFrame'] = 400
df.at[1,'endFrame'] = 400
df.at[2,'endFrame'] = 400
df.at[3,'endFrame'] = 400
df.at[5,'endFrame'] = 400
df.at[7,'endFrame'] = 410
df.at[8,'endFrame'] = 440
df.at[4,'endFrame'] = 430
df.at[10,'endFrame'] = 430
df.at[12,'endFrame'] = 440
df.at[13,'endFrame'] = 425
df.at[14,'endFrame'] = 420
df.at[15,'endFrame'] = 405
df.at[11,'endFrame'] = 420
df.at[16,'endFrame'] = 450
df.at[17,'endFrame'] = 410
df.at[18,'endFrame'] = 430
df.at[19,'endFrame'] = 440
df.at[20,'endFrame'] = 450
df.at[17,'Video'] = 'Night_Rain_High_3'

In [403]:
saveName = fileNames[6].split(".csv")[0]
anna = pd.read_csv(fileNames[6])

In [404]:
anna['mrgTime'] = round(anna['MediaTime']/1000)

In [405]:
# videoPath = "/Users/sonia/Desktop/Projects/EyeCar/FrameObj/"
# savePath = "/Users/sonia/Desktop/Projects/videoObj/"
# for n in Names:
#     listFiles = [f for f in glob.glob(videoPath + n + "_output_*.csv")]
#     combined = pd.concat([pd.read_csv(f) for f in listFiles])
#     combined.to_csv(savePath + n + "_all.csv")

In [406]:
data = {}
for f in glob.glob(savePath+"*.csv"):
    file = pd.read_csv(f)[['box_points', 'frame']].sort_values('frame')
    file['mrgTime'] = round(file['frame']/15)
    name = f.split("Projects/videoObj/")[1].split("_all.csv")[0]
    data[name] = file

In [407]:
newData = {}
for key in data:
    videoData = data[key]
    eegData = anna[anna['StimulusName'] == key]
    tmp = pd.merge(videoData, eegData, on = "mrgTime")
    newData[key] = tmp

In [408]:
finalData = {}
for key in newData:
    tmp = newData[key]
    tmp['hazardousFrame'] = 0
    startFrame = df.loc[df['Video'] == key]['startFrame'].values[0]
    endFrame = df.loc[df['Video'] == key]['endFrame'].values[0]
    tmp.loc[(tmp['frame'] >= startFrame) & (tmp['frame'] <= endFrame), 'hazardousFrame'] = 1
    finalData[key] = tmp

In [409]:
for key in finalData:
    print(key)
    print(finalData[key]['hazardousFrame'].unique())

Day_Rain_High_1
[0 1]
Day_Rain_High_2
[0 1]
Day_Rain_Town_1
[0 1]
Day_Rain_Town_2
[0 1]
Day_Rain_Town_3
[0 1]
Day_Sunny_High_1
[0 1]
Day_Sunny_High_2
[0 1]
Day_Sunny_Town_1
[0 1]
Day_Sunny_Town_3
[0 1]
Day_Sunny_Town_4
[]
Night_Rain_High_1
[1]
Night_Rain_High_2
[0 1]
Night_Rain_High_3
[0 1]
Night_Rain_High_4
[0 1]
Night_Rain_Town_2
[0 1]
Night_Rain_Town_3
[0 1]
Night_Sunny_High_1
[0 1]
Night_Sunny_High_2
[0 1]
Night_Sunny_Town_1
[0 1]
Night_Sunny_Town_2
[0 1]


In [410]:
rmCols = ['mrgTime', 'Unnamed: 0', 'frame', 'MediaTime', 'TimeSignal']
hazardousData = pd.DataFrame()
for key in finalData:
    tmp = finalData[key]
    tmp.drop(rmCols, 1)
    hazardousData = pd.concat([hazardousData, tmp], axis = 0)
hazardousData.to_csv(savePath + "/observer/" + saveName + "_hazardous.csv")

In [411]:
hazardousData.columns

Index(['box_points', 'frame', 'mrgTime', 'Unnamed: 0', 'Name', 'Age', 'Gender',
       'StimulusName', 'EventSource', 'Timestamp',
       ...
       'F8 (Epoc)', 'AF4 (Epoc)', 'Gyro-X (Epoc)', 'Gyro-Y (Epoc)',
       'SystemTimestamp (Epoc)', 'Heart Rate PPG  (Beats/min) (Shimmer)',
       'IBI PPG  (mSecs) (Shimmer)', 'GSR CAL (¬µSiemens) (Shimmer)',
       'Packet reception rate RAW (no units) (Shimmer)', 'hazardousFrame'],
      dtype='object', length=107)